In [ ]:

import vtk
import os
import numpy as np
import itertools
import math, random
random.seed = 42

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import scipy.spatial.distance
# import plotly.graph_objects as go
# import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix



In [ ]:

#load 3d moded
path = "../input/modelnet10-princeton-3d-object-dataset/ModelNet10"

folders = [dir for dir in sorted(os.listdir(path)) if os.path.isdir(path/dir)]
classes = {folder: i for i, folder in enumerate(folders)};
classes



In [ ]:
class Aneuxmodel_Dataset(Dataset):

    def __init__(self, df, root = "", transform = None,train = False):

        self.root = root
        self.transform = transform
        self.training = train
        self.df = df
        self.label = []
        self.model_file = []
        self.training_data_load()
        self.my_device = "cuda:0"
            
            
    def training_data_load(self):
        self.label = []
        self.model_file = []
        all_imgs_dir = os.listdir(self.root)
        transform_im= transforms.ToTensor()
        for model in all_imgs_dir:
                    
            reader = vtk.vtkXMLPolyDataReader()
            reader.SetFileName(model)
            reader.Update()
            polydata = reader.GetOutput()
            points = polydata.GetPoints()
            polygons = polydata.GetPolys()

            self.label.append(points)
        
        return True
        # self.org_imgs = np.array(self.org_imgs)
        # self.total_imgs = torch.from_numpy(np.array(self.total_imgs)) 
        # self.label = torch.from_numpy(np.array(self.label))  
        # return True

    
    def __getitem__(self, index):
        
        """ Returns one data pair (image and target caption). """
        image = self.org_imgs[index]
        #print(type(image))
        if self.transform is not None:      
            image= self.transform(image)
        return image,self.label[index]

    def __len__(self):
                                
        return len(self.org_imgs)

# PointNet model

In [ ]:
from torch import Tensor
from torch.nn import Sequential, Linear, ReLU,PointNet,PointNetLayer


class PointNet(torch.nn.Module):
    def __init__(self,dataset):
        super().__init__()

        self.conv1 = PointNetLayer(3, 32)
        self.conv2 = PointNetLayer(32, 32)
        self.classifier = Linear(32, dataset.num_classes)

    def forward(self,
        pos: Tensor,
        edge_index: Tensor,
        batch: Tensor,
    ) -> Tensor:

        # Perform two-layers of message passing:
        h = self.conv1(h=pos, pos=pos, edge_index=edge_index)
        h = h.relu()
        h = self.conv2(h=h, pos=pos, edge_index=edge_index)
        h = h.relu()

        # Global Pooling:
        # h = global_max_pool(h, batch)  # [num_examples, hidden_channels]

        # Classifier:
        return self.classifier(h)

# PointNet ++